# RSI Mean Reversion (Managed)

Hypothèse: en conditions de survente (RSI bas) et contact de la bande basse de Bollinger, le prix a tendance à revenir vers la moyenne. On évite de trader contre des tendances haussières fortes pour réduire les faux signaux.


## Indicateurs
- SMA200 (filtre de régime long terme)
- SMA50 (filtre de régime moyen terme pour éviter les tendances fortes)
- RSI(14) avec seuil de survente (30) et sortie vers 50
- Bandes de Bollinger (20, 2.0) pour détecter les excès
- ATR pour la gestion du risque via ManagedStrategy (SL/TP)


In [1]:
from pathlib import Path
import sys

def find_project_root(start=None):
    start = Path.cwd() if start is None else Path(start)
    for candidate in [start] + list(start.parents):
        if (candidate / 'utils').exists() and (candidate / 'backtesting').exists():
            return candidate
    return start

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('Project root:', PROJECT_ROOT)


Project root: c:\Users\saill\Desktop\t_project


In [6]:
# Configuration notebook (à adapter si besoin)
TICKER = 'SPY'
START_DATE = '2015-01-01'
END_DATE = '2025-01-01'

TIMEFRAME_DAILY = '1d'
TIMEFRAME_INTRADAY = '1h'
RESAMPLE_TO = '4H'  # resample intraday feed to this timeframe

RISK_PCT = 0.02  # 2% risk per trade
DEFAULT_STOP_DISTANCE = 0.03  # used by FixedFractionalSizer

# Paramètres RSI Mean Reversion
TREND_LONG_PERIOD = 200
AVOID_STRONG_TREND = False
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_EXIT = 50
BB_PERIOD = 20
BB_DEV = 2.0
REENTRY_COOLDOWN_BARS = 5
ATR_PERIOD = 14


In [7]:
import pandas as pd

from utils.data_manager import DataManager
from utils.data_processor import resample_data

dm = DataManager()
df_daily = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_DAILY)
df_intraday = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_INTRADAY)

if df_intraday is not None and not df_intraday.empty:
    df_4h = resample_data(df_intraday, rule=RESAMPLE_TO)
else:
    df_4h = pd.DataFrame()
    print('Intraday data unavailable; continuing without 4H resample.')

print(f'Daily bars: {len(df_daily)}')
if not df_4h.empty:
    print(f'4H bars: {len(df_4h)}')


$SPY: possibly delisted; no price data found  (1h 2023-11-06 -> 2025-11-03) (Yahoo error = "1h data not available for startTime=1699246800 and endTime=1762146000. The requested range must be within the last 730 days.")
2025-11-05 21:05:04 - utils.data_manager - ERROR - Impossible d'obtenir des données pour SPY.


Intraday data unavailable; continuing without 4H resample.
Daily bars: 2516


In [8]:
# Backtest - RsiMeanReversionManagedStrategy
import backtrader as bt
from backtesting.engine import BacktestEngine
from risk_management.position_sizing import FixedFractionalSizer
from strategies.implementations.rsi_mean_reversion_managed_strategy import RsiMeanReversionManagedStrategy
from utils.config_loader import get_settings

engine = BacktestEngine()
# Analyzers additionnels (en plus de ceux ajoutés par défaut)
engine.cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')
engine.cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')

# Position sizing
engine.add_sizer(FixedFractionalSizer, risk_pct=RISK_PCT, stop_distance=DEFAULT_STOP_DISTANCE)

# Flux de données
engine.add_data(df_daily, name='data_1d')
if 'df_4h' in globals() and not df_4h.empty:
    engine.add_data(df_4h, name='data_4h')

# Paramétrage de la stratégie
engine.add_strategy(
    RsiMeanReversionManagedStrategy,
    trend_long_period=TREND_LONG_PERIOD,
    avoid_strong_trend=AVOID_STRONG_TREND,
    rsi_period=RSI_PERIOD,
    rsi_oversold=RSI_OVERSOLD,
    rsi_exit=RSI_EXIT,
    bb_period=BB_PERIOD,
    bb_dev=BB_DEV,
    use_invalidation=True,
    reentry_cooldown_bars=REENTRY_COOLDOWN_BARS,
    atr_period=ATR_PERIOD,
)

# Exécution
results = engine.run()
strategy_instance = results[0]

# Résumé des résultats
settings = get_settings()
initial_capital = settings.get('backtest', {}).get('initial_capital', 10000.0)
from scripts.run_backtest import print_results
print_results(results, initial_capital, df_daily)



RÉSULTATS DU BACKTEST

📊 Période: 2015-01-02 à 2024-12-31
📊 Nombre de bougies: 2516

💰 PERFORMANCE
   Capital Initial:              10,000.00 €
   Capital Final:                 8,927.98 €
   P&L:                          -1,072.02 € (-10.72%)
   Retour Total:                    -11.34%
   Retour Moyen (annuel):            -0.00%

📈 TRADES
   Nombre Total:                        11
   Trades Gagnants:                      5
   Trades Perdants:                      6
   Win Rate:                        45.45%
   Gain Moyen:                      204.86 €
   Perte Moyenne:                  -349.39 €

📉 RISQUE
   Sharpe Ratio:           -0.4783354451820341
   Max Drawdown:                    16.04%



## Optionnel: Visualisation Backtrader
La commande ci-dessous produit un graphique via matplotlib si disponible.


In [9]:
# engine.plot(style='candlestick')
